In [1]:
from platform import python_version
print('A versão usada foi', python_version())

A versão usada foi 3.9.12


In [1]:
# Importando e inicializando o findspark
import findspark
findspark.init()

In [2]:
 #Imports 
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.ml.feature import StringIndexer
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

## Carregando os dados

In [3]:
# Criando o spark context

sc = SparkContext(appName='Note2')

23/04/09 17:27:27 WARN Utils: Your hostname, mor-Inspiron-3501 resolves to a loopback address: 127.0.1.1; using 192.168.15.175 instead (on interface wlp0s20f3)
23/04/09 17:27:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/09 17:27:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/04/09 17:27:28 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
sc.setLogLevel('ERROR')

In [5]:
# Spark Session - usada quando se trabalha com Dataframes no Spark
spark_sesseion = SparkSession.builder.master('local').getOrCreate()

In [6]:
# Carregando os dados e gerando um RDD
dados_times_futebol = sc.textFile('dataset2.csv')

In [7]:
# Colocando o RDD em cache. Esse processo otimiza a performance.
dados_times_futebol.cache()

dataset2.csv MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [9]:
dados_times_futebol.count()

151

In [8]:
dados_times_futebol.take(5)

['media_faltas_sofridas,media_faltas_recebidas,media_cartoes_recebidos,media_chutes_a_gol,resultado',
 '4.8,3,1.4,0.3,vitoria',
 '5.1,3.8,1.6,0.2,vitoria',
 '4.6,3.2,1.4,0.2,vitoria',
 '5.3,3.7,1.5,0.2,vitoria']

In [9]:
# Removendo a primeira linha do arquivo (cabeçalho)
dados_times_futebol2 = dados_times_futebol.filter(lambda x: 'media_faltas_sofridas' not in x)
dados_times_futebol2.count()

150

## Limpeza dos Dados

In [10]:
# Seprando as colunas
dados_times_futebol3 = dados_times_futebol2.map(lambda x: x.split(','))

In [11]:
dados_times_futebol3.take(5)

[['4.8', '3', '1.4', '0.3', 'vitoria'],
 ['5.1', '3.8', '1.6', '0.2', 'vitoria'],
 ['4.6', '3.2', '1.4', '0.2', 'vitoria'],
 ['5.3', '3.7', '1.5', '0.2', 'vitoria'],
 ['5.1', '3.5', '1.4', '0.2', 'vitoria']]

In [12]:
# Mapeando as colunas
# Isso é necessário porque o Spark não carregou o Schemma. Para o rdd não importa.
# Mas para o spark dataframe isso é necessário!
dados_times_futebol4 = dados_times_futebol3.map(lambda p: Row(media_faltas_sofridas = float(p[0]), 
                                                              media_faltas_recebidas = float(p[1]), 
                                                              media_cartoes_recebidos = float(p[2]), 
                                                              media_chutes_a_gol = float(p[3]), 
                                                              resultado = p[4]))

In [13]:
# Converte o RDD para DataFrame do Spark
df_time = spark_sesseion.createDataFrame(dados_times_futebol4)
df_time.cache()

DataFrame[media_faltas_sofridas: double, media_faltas_recebidas: double, media_cartoes_recebidos: double, media_chutes_a_gol: double, resultado: string]

In [14]:
df_time.take(5)

[Row(media_faltas_sofridas=4.8, media_faltas_recebidas=3.0, media_cartoes_recebidos=1.4, media_chutes_a_gol=0.3, resultado='vitoria'),
 Row(media_faltas_sofridas=5.1, media_faltas_recebidas=3.8, media_cartoes_recebidos=1.6, media_chutes_a_gol=0.2, resultado='vitoria'),
 Row(media_faltas_sofridas=4.6, media_faltas_recebidas=3.2, media_cartoes_recebidos=1.4, media_chutes_a_gol=0.2, resultado='vitoria'),
 Row(media_faltas_sofridas=5.3, media_faltas_recebidas=3.7, media_cartoes_recebidos=1.5, media_chutes_a_gol=0.2, resultado='vitoria'),
 Row(media_faltas_sofridas=5.1, media_faltas_recebidas=3.5, media_cartoes_recebidos=1.4, media_chutes_a_gol=0.2, resultado='vitoria')]

In [15]:
# Criando um índice numérico para a coluna de label target
string_indexer = StringIndexer(inputCol='resultado', outputCol='idx_resultado')

In [16]:
# Treinando o string indexer
s_i_model = string_indexer.fit(df_time)

In [18]:
# Aplicando o string indexer já treinado
df_time_final = s_i_model.transform(df_time)

In [20]:
df_time_final.select('resultado', 'idx_resultado').distinct().collect()

[Row(resultado='derrota', idx_resultado=0.0),
 Row(resultado='vitoria', idx_resultado=2.0),
 Row(resultado='empate', idx_resultado=1.0)]

## Análise Exploratória de Dados

In [21]:
# Estatística descritiva:
df_time_final.describe().show()

+-------+---------------------+----------------------+-----------------------+------------------+---------+------------------+
|summary|media_faltas_sofridas|media_faltas_recebidas|media_cartoes_recebidos|media_chutes_a_gol|resultado|     idx_resultado|
+-------+---------------------+----------------------+-----------------------+------------------+---------+------------------+
|  count|                  150|                   150|                    150|               150|      150|               150|
|   mean|    5.843333333333332|    3.0573333333333337|      3.758000000000001|1.1993333333333331|     null|               1.0|
| stddev|   0.8280661279778625|   0.43586628493669793|     1.7652982332594667|0.7622376689603465|     null|0.8192319205190404|
|    min|                  4.3|                   2.0|                    1.0|               0.1|  derrota|               0.0|
|    max|                  7.9|                   4.4|                    6.9|               2.5|  vitoria|    

In [22]:
# Correlação entre as variáveis

for i in df_time_final.columns:
    if not(isinstance(df_time_final.select(i).take(1)[0][0], str)):
        print("Correlação da variável idx_resultado com", i, df_time_final.corr('idx_resultado', i))

Correlação da variável idx_resultado com media_faltas_sofridas -0.460039156500237
Correlação da variável idx_resultado com media_faltas_recebidas 0.6183715308237437
Correlação da variável idx_resultado com media_cartoes_recebidos -0.6492418307641741
Correlação da variável idx_resultado com media_chutes_a_gol -0.5803770334306263
Correlação da variável idx_resultado com idx_resultado 1.0


## Pré-Processamento dos Dados

In [31]:
# Criando um LabeledPoint (target, Vector[features])
# Remove colunas não relevantes para o modelo ou com baixa correlação

def transformaVar(row):
    obj = (row['resultado'], row['idx_resultado'],Vectors.dense(row["media_faltas_sofridas"], 
                                                                  row["media_faltas_recebidas"],
                                                                  row["media_cartoes_recebidos"], 
                                                                  row["media_chutes_a_gol"]))
    return obj

In [32]:
# Aplica a função
df_time_final_RDD = df_time_final.rdd.map(transformaVar)

In [33]:
df_time_final_RDD.take(5)

[('vitoria', 2.0, DenseVector([4.8, 3.0, 1.4, 0.3])),
 ('vitoria', 2.0, DenseVector([5.1, 3.8, 1.6, 0.2])),
 ('vitoria', 2.0, DenseVector([4.6, 3.2, 1.4, 0.2])),
 ('vitoria', 2.0, DenseVector([5.3, 3.7, 1.5, 0.2])),
 ('vitoria', 2.0, DenseVector([5.1, 3.5, 1.4, 0.2]))]

In [34]:
# Converte o RDD para DataFrame
df_spark = spark_sesseion.createDataFrame(df_time_final_RDD,['resultado', 'label', 'features'])

In [35]:
df_spark.cache()

DataFrame[resultado: string, label: double, features: vector]

In [36]:
df_spark.select('resultado', 'label', 'features').show(10)

+---------+-----+-----------------+
|resultado|label|         features|
+---------+-----+-----------------+
|  vitoria|  2.0|[4.8,3.0,1.4,0.3]|
|  vitoria|  2.0|[5.1,3.8,1.6,0.2]|
|  vitoria|  2.0|[4.6,3.2,1.4,0.2]|
|  vitoria|  2.0|[5.3,3.7,1.5,0.2]|
|  vitoria|  2.0|[5.1,3.5,1.4,0.2]|
|  vitoria|  2.0|[4.9,3.0,1.4,0.2]|
|  vitoria|  2.0|[4.7,3.2,1.3,0.2]|
|  vitoria|  2.0|[4.6,3.1,1.5,0.2]|
|  vitoria|  2.0|[5.0,3.6,1.4,0.2]|
|  vitoria|  2.0|[5.4,3.9,1.7,0.4]|
+---------+-----+-----------------+
only showing top 10 rows



In [37]:
# Dados de Treino e de Teste
(dados_treino, dados_teste) = df_spark.randomSplit([0.7,0.3])
print(dados_treino.count())
print(dados_teste.count())

109
41


## Machine Learning

In [38]:
#Cria o objeto
classificador = DecisionTreeClassifier(maxDepth= 2, labelCol='label', featuresCol='features')

In [39]:
# Treina o objeto com dados para criar o modelo
modelo = classificador.fit(dados_treino)

In [40]:
# Hiperparâmetro definido por padrão
modelo.numNodes

5

In [41]:
# Hiperparâmetro definido por nós
modelo.depth

2

In [42]:
# Previsões com dados de teste
prev = modelo.transform(dados_teste)
prev

DataFrame[resultado: string, label: double, features: vector, rawPrediction: vector, probability: vector, prediction: double]

In [43]:
prev.select("resultado", "label", "prediction", "probability").collect()

[Row(resultado='derrota', label=0.0, prediction=0.0, probability=DenseVector([0.9167, 0.0833, 0.0])),
 Row(resultado='derrota', label=0.0, prediction=0.0, probability=DenseVector([0.9167, 0.0833, 0.0])),
 Row(resultado='derrota', label=0.0, prediction=0.0, probability=DenseVector([0.9167, 0.0833, 0.0])),
 Row(resultado='derrota', label=0.0, prediction=0.0, probability=DenseVector([0.9167, 0.0833, 0.0])),
 Row(resultado='derrota', label=0.0, prediction=0.0, probability=DenseVector([0.9167, 0.0833, 0.0])),
 Row(resultado='derrota', label=0.0, prediction=0.0, probability=DenseVector([0.9167, 0.0833, 0.0])),
 Row(resultado='derrota', label=0.0, prediction=0.0, probability=DenseVector([0.9167, 0.0833, 0.0])),
 Row(resultado='derrota', label=0.0, prediction=0.0, probability=DenseVector([0.9167, 0.0833, 0.0])),
 Row(resultado='derrota', label=0.0, prediction=0.0, probability=DenseVector([0.9167, 0.0833, 0.0])),
 Row(resultado='vitoria', label=2.0, prediction=2.0, probability=DenseVector([0.0,

In [44]:
# Avaliando a acurácia
avaliador = MulticlassClassificationEvaluator(predictionCol='prediction',
                                             labelCol='label',
                                             metricName='accuracy')

In [45]:
avaliador.evaluate(prev)

0.9512195121951219

In [46]:
# Resumindo as previsões - Confusion Matrix
prev.groupBy('label', 'prediction').count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  2.0|       2.0|   14|
|  0.0|       0.0|   16|
|  1.0|       1.0|    9|
|  1.0|       0.0|    2|
+-----+----------+-----+



## Fim